In [ ]:
### QCLab: Bernstein–Vazirani Algorithm – Finding the Hidden String

The task is to find a hidden n-bit binary string $s$ encoded in the oracle $O_f$. However, the oracle does not reveal the string directly when queried. Instead, for each query string $x$, it replies with the binary inner product between $x$ and $s$:

$$
f(x) = s \cdot x
     = \bigoplus_{i=1}^n (s_i \cdot x_i) 
     = (s_1 \cdot x_1) \oplus (s_2 \cdot x_2) \oplus \cdots \oplus (s_n \cdot x_n)
$$

Classically, recovering $s$ requires $n$ different queries. The Bernstein–Vazirani algorithm, however, can determine the hidden $n$-bit string $s$ with just **one** query. As shown in the figure, the algorithm prepares the input register in a superposition over all $x$, then applies the oracle as a phase oracle:

$$
|x\rangle \mapsto (-1)^{f(x)} |x\rangle
$$

After applying interference to the oracle’s answers, the measurement reveals the hidden string.

![Bernstein–Vazirani](images/Bernstein–Vazirani.png)
---

### Task
- The program should prompt the user to enter a binary string $s$, which will serve as the hidden string the algorithm must determine.  
- Based on this string, the program should construct an oracle that computes the binary inner product of an input query string $x$ with the hidden string $s$.  
- Incorporate this oracle into a Bernstein–Vazirani quantum circuit that recovers the hidden string $s$ in a single query.

### Expected Output
- A plot of the Bernstein–Vazirani circuit.  
- A histogram of the measurement results.  
- A printout of the measurement results dictionary.  

### Experimentation
- Test the algorithm with different hidden strings $s$ of various lengths (e.g., 3, 4, or 5 bits) and verify that the output matches the input.  
- Observe how the histogram changes when you vary the hidden string.  
- Compare the number of queries required classically versus the single query in the quantum version.  
- Try modifying the oracle construction and confirm that the algorithm still correctly identifies the hidden string.  


In [ ]:
# ====================================================
# QCLab: Bernstein–Vazirani Algorithm
# <QC|CT> qcict.org
# ====================================================

from IPython.display import display

from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram,plot_distribution
from qiskit.visualization import circuit_drawer

def bv_oracle(s):
    """
    Bernstein–Vazirani oracle for secret string s.

    The oracle implements:
        f(x) = s · x
    where '·' is the bitwise dot product modulo 2.

    For each bit s_i = 1 in the secret string:
      - A CNOT gate is applied from input qubit x_i to the target qubit.
      - This flips the target qubit if and only if x_i = 1.

    As a result, the target qubit is flipped for every input qubit position
    where s_i = 1, effectively computing the XOR of the selected input bits.
    """
    n = len(s)
    qc = QuantumCircuit(n+1)

    qc.barrier()  # visual separation from previous circuit steps
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(i, n)  # XOR xi into target if si = 1
    
    qc.barrier()  # visual separation from previous circuit steps
    
    return qc

def bv_circuit(s):
    """
    Builds the Bernstein–Vazirani circuit for the given secret bitstring s.

    The circuit prepares the target (auxiliary) qubit in the |−⟩ state to enable
    bit flipping via the oracle. It sandwiches the oracle between two layers of
    Hadamard gates on the input qubits, and finally measures the input register
    to reveal the hidden string in a single query.
    """
    n = len(s)
    qc = QuantumCircuit(n+1, n)

    # Step 1: Initialize target qubit in |-> state
    qc.x(n)
    qc.h(n)

    # Step 2: Apply Hadamard to all input qubits
    qc.h(range(n))

    # Step 3: Apply oracle
    qc.compose(bv_oracle(s), inplace=True)

    # Step 4: Apply Hadamard to input qubits again
    qc.h(range(n))

    # Step 5: Measure input qubits (ignore target)
    qc.measure(range(n), range(n))

    return qc

# ------------------------------------------------
#                main program
# ------------------------------------------------

# Ask user for the secret bitstring (any length)
while True:
    s = input("Enter a secret bitstring (e.g., 1011) that the algorithm must guess:")
    if len(s) > 0 and all(bit in '01' for bit in s):
        break
    print("Invalid input. Please enter a non-empty string of bits (0 or 1).")

# -- build and run circuit --
qc = bv_circuit(s)
simulator = AerSimulator()
result = simulator.run(qc, shots=1000).result()
counts = result.get_counts(qc)

# -- display --
display(circuit_drawer(qc, style="bw", output="mpl"))
display(plot_histogram(counts))

print("Measurement result:", counts)
